In [1]:
import cantera as ct 
from rmgpy import chemkin
from rmgpy import kinetics

###N = David's R = Nora's
#N = Nora's, R = Davids

In [21]:
def compare_rxn_equations(modelD_reactions, modelN_reactions):
    '''
    Compares the reaction equations of each model. Uses to_cantera() on RMG model to eliminate chemkin_identifier 
    '''

################### mostly taken from diffmodel.py, compare_model_reactions() ###################

    #remove reactions with unknown species
    to_remove = []
    for reactionList in (modelD_reactions, modelN_reactions):
        for reaction in reactionList:
            for side in (reaction.products, reaction.reactants):
                for species in side:
                    if not species.molecule:
                        to_remove.append((reactionList, reaction))
                        print("Removing reaction {!r} that had unidentified species {!r}".format(reaction, species))
                        break
    for reactionList, reaction in to_remove:
        reactionList.remove(reaction)
    print(f'Following reactions removed because they have unknown species: {to_remove}')
    
    #find the common reactions and the unique reactions
    common_reactions = []
    unique_reactionsD = []
    unique_reactionsN = []
    copy_of_modelN_reactions = modelN_reactions.copy() #list 1
    copy_of_modelD_reactions = modelD_reactions.copy() #list 2

    
    for rxnN in copy_of_modelN_reactions:
        for rxnD in copy_of_modelD_reactions:  # make a copy so you don't remove from the list you are iterating over
            if rxnN.is_isomorphic(rxnD):
                common_reactions.append([rxnN, rxnD])
                #common_reactions_equations.append([str(rxnN), str(rxnD)])
                # Remove reaction 2 from being chosen a second time.
                # Let each reaction only appear only once in the diff comparison.
                # Otherwise this miscounts number of reactions in model 2.
                copy_of_modelD_reactions.remove(rxnD)
 
    
    for rxnN in copy_of_modelN_reactions:
        for rN, rD in common_reactions:
            if rxnN is rN:
                break
        else:
            unique_reactionsN.append(rxnN)
    for rxnD in copy_of_modelD_reactions:
        for rN, rD in common_reactions:
            if rxnD is rD:
                break
        else:
            unique_reactionsD.append(rxnD)
            
    different_kinetics_reactions = []
            
    for rN, rD in common_reactions: 
        if (rN.kinetics.is_identical_to(rD.kinetics)==False) and (rN.kinetics.is_similar_to(rD.kinetics)==False):
            different_kinetics_reactions.append(rD)
            
    return different_kinetics_reactions, common_reactions, unique_reactionsD, unique_reactionsN
    
    
    

def details(different_kinetics_reactions, common_reactions, unique_reactionsD, unique_reactionsN):
    
    '''
    Takes rmg objects and extracts readable info (equations, indexes, etc)
    '''
    
    diff_kin_equations = [str(i) for i in different_kinetics_reactions]
    comm_equations = [str(i) for i in common_reactions]
    
    uniq_rxns_eq_D = [str(i) for i in unique_reactionsD]

    uniq_rxns_eq_N = [str(i) for i in unique_reactionsN]
    
    return diff_kin_equations, comm_equations, uniq_rxns_eq_D, uniq_rxns_eq_N 
    
    
    
    
    
    
    
def execute(full_pathD, chemkinD, dictionaryD, full_pathN, chemkinN, dictionaryN): 
    '''
    loads in chemkin files and runs commands to execute the file
    '''
    full_pathN = full_pathN
    inp_pathN = full_pathN + chemkinN
    dict_pathN = full_pathN + dictionaryN
    modelN_species, modelN_reactions = chemkin.load_chemkin_file(inp_pathN, dict_pathN)

    full_pathD = full_pathD
    inp_pathD = full_pathD + chemkinD
    dict_pathD = full_pathD + dictionaryD
    modelD_species, modelD_reactions = chemkin.load_chemkin_file(inp_pathD, dict_pathD)
    
    different_kinetics_reactions, common_reactions, unique_reactionsD, unique_reactionsN = compare_rxn_equations(modelD_reactions, modelN_reactions)
    diff_kin_equations, comm_equations, uniq_rxns_eq_D, uniq_rxns_eq_N  = details(different_kinetics_reactions, common_reactions, unique_reactionsD, unique_reactionsN)
    return different_kinetics_reactions, diff_kin_equations, common_reactions, comm_equations, unique_reactionsD, uniq_rxns_eq_D, unique_reactionsN, uniq_rxns_eq_N


different_kinetics_reactions, diff_kin_equations, common_reactions, comm_equations, unique_reactionsD, uniq_rxns_eq_D, unique_reactionsN, uniq_rxns_eq_N = execute(
    "/work/westgroup/nora/Code/projects/halogens/refrigerants/singles/Burgess_Comments/compare/models/Nora/2_BTP_seed/chemkin/",
    "copies/copy_chem_130.inp",
    "species_dictionary.txt",
    "/work/westgroup/nora/Code/projects/halogens/refrigerants/singles/Burgess_Comments/compare/models/David/2-BTP/chemkin/",
    "copies/copy_chem.inp",
    "species_dictionary.txt"
    )

#N = David's R = Nora's
#1008 common reactions 
#1290 unique reactions in Davids, 292 unique reactions in Nora's
    

Following reactions removed because they have unknown species: []


In [39]:
common_reaction_index_D = []
common_reaction_index_N = []
for rN, rD in common_reactions: 
    common_reaction_index_D.append(rD.index)
    common_reaction_index_N.append(rN.index)



[LibraryReaction(index=2, reactants=[Species(index=7, label="O", thermo=NASA(polynomials=[NASAPolynomial(coeffs=[2.5,-1.84483e-15,2.71425e-18,-1.30028e-21,1.91033e-25,29230.2,5.12616], Tmin=(100,'K'), Tmax=(3598.68,'K')), NASAPolynomial(coeffs=[2.5,-2.82485e-12,1.07037e-15,-1.78888e-19,1.11248e-23,29230.2,5.12616], Tmin=(3598.68,'K'), Tmax=(5000,'K'))], Tmin=(100,'K'), Tmax=(5000,'K')), molecule=[Molecule(smiles="[O]")], molecular_weight=(15.9994,'amu')), Species(index=8, label="H2", thermo=NASA(polynomials=[NASAPolynomial(coeffs=[3.43536,0.000212711,-2.78626e-07,3.40268e-10,-7.76035e-14,-1031.36,-3.90842], Tmin=(100,'K'), Tmax=(1959.07,'K')), NASAPolynomial(coeffs=[2.78817,0.000587632,1.59015e-07,-5.52748e-11,4.34317e-15,-596.149,0.11269], Tmin=(1959.07,'K'), Tmax=(5000,'K'))], Tmin=(100,'K'), Tmax=(5000,'K')), molecule=[Molecule(smiles="[H][H]")], molecular_weight=(2.01594,'amu'))], products=[Species(index=6, label="H", thermo=NASA(polynomials=[NASAPolynomial(coeffs=[2.5,-1.84483e-15